In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(14993, 128, 128, 3)

In [8]:
y.shape

(14993, 8)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(11994, 128, 128, 3)

In [11]:
X_test.shape

(2999, 128, 128, 3)

In [12]:
y_train.shape

(11994, 8)

In [13]:
y_test.shape

(2999, 8)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

8

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 11:29:27.937229: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


375/375 [==============================] - 81s 215ms/step - loss: 0.6593 - accuracy: 0.7623 - val_loss: 0.6310 - val_accuracy: 0.7706
Epoch 2/10
375/375 [==============================] - 85s 228ms/step - loss: 0.2470 - accuracy: 0.9145 - val_loss: 0.2707 - val_accuracy: 0.9130
Epoch 3/10
375/375 [==============================] - 87s 232ms/step - loss: 0.1486 - accuracy: 0.9501 - val_loss: 0.0995 - val_accuracy: 0.9677
Epoch 4/10
375/375 [==============================] - 87s 233ms/step - loss: 0.1184 - accuracy: 0.9583 - val_loss: 0.0736 - val_accuracy: 0.9747
Epoch 5/10
375/375 [==============================] - 86s 229ms/step - loss: 0.0918 - accuracy: 0.9697 - val_loss: 0.2016 - val_accuracy: 0.9320
Epoch 6/10
375/375 [==============================] - 87s 231ms/step - loss: 0.0679 - accuracy: 0.9760 - val_loss: 0.5079 - val_accuracy: 0.8403
Epoch 7/10
375/375 [==============================] - 88s 235ms/step - loss: 0.0717 - accuracy: 0.9747 - val_loss: 0.2610 - val_accuracy: 0.9

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 97.10%


In [19]:
cnn_scores

[0.08717964589595795, 0.9709903597831726]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

94/94 [==============================] - 12s 117ms/step
Accuracy: 97.10%
Precision: 97.20%
Recall: 97.10%
F1 score: 97.10%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-apel+anggur.csv', index=False)

+------+----------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                   | True Labels                                | Predicted Labels                           |
|------+----------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|    0 | cb340afd-cfe7-47f7-84c6-28c8d4dd3166___FAM_L.Blight 0803_flipLR.JPG        | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) |
|    1 | 3e19a6c4-e5d8-4649-b403-7a8fd36b88e2___RS_HL 7533.JPG                      | Apple___healthy                            | Apple___healthy                            |
|    2 | 18df67df-8153-4b09-87a1-8b3d3f82e86b___FAM_B.Msls 3949.JPG                 | Grape___Esca_(Black_Measles)      

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-apel+anggur.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                   | True Labels                                | Predicted Labels                           |
|------+----------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|   27 | 307abcdd-96f7-404a-9032-2ce8de2fde48___JR_FrgE.S 8781_90deg.JPG            | Apple___Black_rot                          | Apple___healthy                            |
|   55 | 1a4e1884-ab1a-4fe9-afe2-610ae9aa1162___JR_FrgE.S 2826_90deg.JPG            | Apple___Black_rot                          | Apple___Cedar_apple_rust                   |
|  107 | 97c0552c-1cd6-4813-9608-3735b9867f98___Mt.N.V_HL 6046_90deg.JPG            | Grape___healthy

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

375/375 [==============================] - 51s 135ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 604.3236448764801 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

94/94 [==============================] - 11s 113ms/step
Random Forest Classifier Accuracy: 98.53%
Random Forest Classifier Precision: 0.99
Random Forest Classifier Recall: 0.99
Random Forest Classifier F1 Score: 0.99


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-apel+anggur.csv', index=False)

+------+----------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                   | True Labels                                | Predicted Labels                           |
|------+----------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|    0 | cb340afd-cfe7-47f7-84c6-28c8d4dd3166___FAM_L.Blight 0803_flipLR.JPG        | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) |
|    1 | 3e19a6c4-e5d8-4649-b403-7a8fd36b88e2___RS_HL 7533.JPG                      | Apple___healthy                            | Apple___healthy                            |
|    2 | 18df67df-8153-4b09-87a1-8b3d3f82e86b___FAM_B.Msls 3949.JPG                 | Grape___Esca_(Black_Measles)      

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-apel+anggur.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                   | True Labels                                | Predicted Labels                           |
|------+----------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|  114 | eca2a279-0f7f-40a7-b2c4-a9bc500f4f5b___FREC_Scab 3015_90deg.JPG            | Apple___Apple_scab                         | Apple___Black_rot                          |
|  181 | 198fbad6-0192-4263-96c8-74c330667d61___Mt.N.V_HL 9032.JPG                  | Grape___healthy                            | Apple___Black_rot                          |
|  207 | c18a2d5a-4160-48de-8e9e-7ccc047aeddc___JR_FrgE.S 8805.JPG                  | Apple___Black_r